# Current Topics and Projects
## 2. Perspective toxicity score

In [1]:
'''
This is the fourth file to be runned.
This file requests the perspective api scores and saves them to the data.
This file takes apprx. 14h tu runn with not extention
We got a request extention of 20 requests per second.
'''

import json
import requests 
import time
import pandas as pd
import csv
from tqdm import tqdm

In [4]:
# Read the CSV file and create a DataFrame
df = pd.read_csv('data/raw/comments.csv')


14373

In [5]:
# Function to send text to Perspective API and get scores
# dependent on your api key you have to take the first function which only allows one request per second
def get_perspective_scores(text):
    '''
    This function retrieves the api scores and waits 1 sec for each request
    '''
    # Replace 'api_key' with your actual Perspective API key
    api_key = ""
    url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze"
    data = {
        "comment": {"text": text},
        "requestedAttributes": {
            "TOXICITY": {},
            "SEVERE_TOXICITY": {},
            "IDENTITY_ATTACK": {},
            "INSULT": {},
            "THREAT": {}},
        "languages": ["de"]
    }
    
    try:
        response = requests.post(
            url,
            params={"key": api_key},
            headers={"Content-Type": "application/json"},
            json=data
        )
    
        scores = {}

        if response.status_code == 200:
            response_data = json.loads(response.text)
            attrscores = response_data['attributeScores']
            for attribute in attrscores:
                score = attrscores[attribute]['summaryScore']['value']
                scores[attribute.lower()] = score

        else:
            print("Error:", response.status_code, response.text)   
    except requests.exceptions.RequestException as e:
        print("Error:", str(e))

    return scores

In [6]:
# Function to send text to Perspective API and get scores
def get_perspective_scores(text):
    '''
    This function retrieves the api scores and waits 1 sec for each 20 request
    '''
    # Replace 'api_key' with your actual Perspective API key
    api_key = ""
    url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze"
    data = {
        "comment": {"text": text},
        "requestedAttributes": {
            "TOXICITY": {},
            "SEVERE_TOXICITY": {},
            "IDENTITY_ATTACK": {},
            "INSULT": {},
            "THREAT": {}
        },
        "languages": ["de"]
    }
    
    # Counter variables
    request_count = 0
    sleep_interval = 20
    
    try:
        response = requests.post(
            url,
            params={"key": api_key},
            headers={"Content-Type": "application/json"},
            json=data
        )
        
        request_count += 1
        
        # Check if the request count reaches the limit
        if request_count % sleep_interval == 0:
            time.sleep(1)  # Sleep for 1 second
        
        scores = {}

        if response.status_code == 200:
            response_data = json.loads(response.text)
            attrscores = response_data['attributeScores']
            for attribute in attrscores:
                score = attrscores[attribute]['summaryScore']['value']
                scores[attribute.lower()] = score

        else:
            print("Error:", response.status_code, response.text)   
    except requests.exceptions.RequestException as e:
        print("Error:", str(e))

    return scores

In [7]:
# Use tqdm to display the progress bar
for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    text = str(row['text'])
    scores = get_perspective_scores(text)

    # Add scores as new columns
    for attribute, score in scores.items():
        column_name = attribute
        df.loc[i, column_name] = score
        

Processing: 100%|██████████| 14373/14373 [16:50<00:00, 14.22it/s]


In [12]:
df.to_csv('data/raw/comments_scores.csv', index=False)